### Advanced Data Transformation with Polars

The examples below use `.head()` to reduce the output to a few rows and take up less space.
If you want the full output, remove `.head()` from the code.

This notebooked is divided into sections. If your code editor supports it, you can use the **Outline** functionality to easily go to the code section you are interested in.

For more details on Polars functions, check out the Polars API reference: https://pola-rs.github.io/polars/py-polars/html/reference/index.html

In [1]:
import polars as pl

# Configure the number of characters to show for each string column
pl.Config.set_fmt_str_lengths(50)

polars.config.Config

#### List operations

In [2]:
data = {
    "branch": ['A', 'B', 'C', 'D'],
    "employees": [['John', 'Lisa', 'Sam'], ['Lisa', 'Alex'], ['Sam', 'Alex', 'John'], ['Lisa', 'Sam']],
    "weekly_sales": [[1200, 2500, 700], [1500, 2700], [1800, 2000, 900], [1700, 2100]],
    "stock_items": [['Item1', 'Item2'], ['Item1', 'Item3'], ['Item2', 'Item3'], ['Item1', 'Item2', 'Item3']],
    "customer_rating": [[3, 4, 4], [5, 4], [4, 3, 3], [5, 5]]
}

df_retail = pl.DataFrame(data)

df_retail

branch,employees,weekly_sales,stock_items,customer_rating
str,list[str],list[i64],list[str],list[i64]
"""A""","[""John"", ""Lisa"", ""Sam""]","[1200, 2500, 700]","[""Item1"", ""Item2""]","[3, 4, 4]"
"""B""","[""Lisa"", ""Alex""]","[1500, 2700]","[""Item1"", ""Item3""]","[5, 4]"
"""C""","[""Sam"", ""Alex"", ""John""]","[1800, 2000, 900]","[""Item2"", ""Item3""]","[4, 3, 3]"
"""D""","[""Lisa"", ""Sam""]","[1700, 2100]","[""Item1"", ""Item2"", ""Item3""]","[5, 5]"


In [3]:
# join the employees working in each branch in a single string

df_retail.select(
    'employees',
    employees_joined = pl.col("employees").list.join(', ')
    )

employees,employees_joined
list[str],str
"[""John"", ""Lisa"", ""Sam""]","""John, Lisa, Sam"""
"[""Lisa"", ""Alex""]","""Lisa, Alex"""
"[""Sam"", ""Alex"", ""John""]","""Sam, Alex, John"""
"[""Lisa"", ""Sam""]","""Lisa, Sam"""


In [4]:
# check which branch has Item1 in stock

df_retail.select(
    'branch',
    'stock_items',
    has_item1 = pl.col("stock_items").list.contains("Item1")
    )

branch,stock_items,has_item1
str,list[str],bool
"""A""","[""Item1"", ""Item2""]",true
"""B""","[""Item1"", ""Item3""]",true
"""C""","[""Item2"", ""Item3""]",false
"""D""","[""Item1"", ""Item2"", ""Item3""]",true


In [5]:
# count how many ratings of 5 each branch has

df_retail.select(
    'customer_rating',
    count_ratings_of_5 = pl.col("customer_rating").list.count_match(5)
    )

customer_rating,count_ratings_of_5
list[i64],u32
"[3, 4, 4]",0
"[5, 4]",1
"[4, 3, 3]",0
"[5, 5]",2


In [6]:
# I can visualize the maximum value using max

df_retail.select(
    pl.col('weekly_sales').list.max()
)

weekly_sales
i64
2500
2700
2000
2100


In [7]:
# I can only the unique values

df_retail.select(
    pl.col('customer_rating').list.unique()
)

customer_rating
list[i64]
"[3, 4]"
"[4, 5]"
"[3, 4]"
[5]


In [8]:
# I can sort the values, and take the second biggest

df_retail.select(
    pl.col('weekly_sales').list.sort().list.take(1)
)

weekly_sales
list[i64]
[1200]
[2700]
[1800]
[2100]


In [9]:
# for functions not available in the list namespace, I can use eval and use all expressions
# inside pl.eval, I can identify the list with pl.element()
# for example, I can rank the sales

df_retail.select(
    'weekly_sales',
    ranked_sales = 
        pl.col('weekly_sales').list.eval(
            pl.element().rank(method="dense", descending=True))
    )


weekly_sales,ranked_sales
list[i64],list[u32]
"[1200, 2500, 700]","[2, 1, 3]"
"[1500, 2700]","[2, 1]"
"[1800, 2000, 900]","[2, 1, 3]"
"[1700, 2100]","[2, 1]"


In [10]:
# using pl.eval I can filter for sales equal or bigger than 2500
# note that here I am using pl.element() twice, the first time to identify the list, the second time to identify the element

df_retail.select(
    'weekly_sales',
    filtered_sales = 
        pl.col('weekly_sales').list.eval(
            pl.element().filter(pl.element()>= 2500))
)

weekly_sales,filtered_sales
list[i64],list[i64]
"[1200, 2500, 700]",[2500]
"[1500, 2700]",[2700]
"[1800, 2000, 900]",[]
"[1700, 2100]",[]


#### String operations

In [11]:
# Data
data = {
    "customer_id": ["C001", "C002", "C003", "C004", "C005"],
    "email": ["john.doe@example.com", "sarah.connor@shopnow.com", "jack.sparrow@onlineshopping.net", "harry.potter@buyonline.org", "lisa.simpson@webshop.com"],
    "feedback": [
        "The #PRODUCT-X1 is AMAZING. I love it.",
        "Didn't like #product-x2, poor quality. #Product-x1 is better.",
        "The #product-x1 is fantastic. Super #happy with it.",
        "The service was bad. Not happy with #PRODUCT-M1. Just as bad as #PRODUCT-M2.",
        "Love #PRODUCT-X1, worth every penny!",
    ],
    "product_id": ["P001", "P002", "P001", "M001", "P001"],
    "order_date": ["2023-06-01", "2023-06-02", "2023-06-03", "2023-06-04", "2023-06-05"],
    "transaction_details": ['{"price": 200.0, "shipping": {"type": "express", "cost": 20.0}}',
        '{"price": 150.0, "shipping": {"type": "standard", "cost": 10.0}}',
        '{"price": 200.0, "shipping": {"type": "express", "cost": 20.0}}',
        '{"price": 250.0, "shipping": {"type": "standard", "cost": 10.0}}',
        '{"price": 200.0, "shipping": {"type": "express", "cost": 20.0}}'],
}

# Create DataFrame
df_feedback = pl.DataFrame(data)

df_feedback

customer_id,email,feedback,product_id,order_date,transaction_details
str,str,str,str,str,str
"""C001""","""john.doe@example.com""","""The #PRODUCT-X1 is AMAZING. I love it.""","""P001""","""2023-06-01""","""{""price"": 200.0, ""shipping"": {""type"": ""express"", …"
"""C002""","""sarah.connor@shopnow.com""","""Didn't like #product-x2, poor quality. #Product-x…","""P002""","""2023-06-02""","""{""price"": 150.0, ""shipping"": {""type"": ""standard"",…"
"""C003""","""jack.sparrow@onlineshopping.net""","""The #product-x1 is fantastic. Super #happy with i…","""P001""","""2023-06-03""","""{""price"": 200.0, ""shipping"": {""type"": ""express"", …"
"""C004""","""harry.potter@buyonline.org""","""The service was bad. Not happy with #PRODUCT-M1. …","""M001""","""2023-06-04""","""{""price"": 250.0, ""shipping"": {""type"": ""standard"",…"
"""C005""","""lisa.simpson@webshop.com""","""Love #PRODUCT-X1, worth every penny!""","""P001""","""2023-06-05""","""{""price"": 200.0, ""shipping"": {""type"": ""express"", …"


In [12]:
# to lowercase

df_feedback.select(
    pl.col('feedback').str.to_lowercase(),
)

feedback
str
"""the #product-x1 is amazing. i love it."""
"""didn't like #product-x2, poor quality. #product-x…"
"""the #product-x1 is fantastic. super #happy with i…"
"""the service was bad. not happy with #product-m1. …"
"""love #product-x1, worth every penny!"""


In [13]:
# filter for rows containing 'service' or 'quality'

df_feedback.filter(
    pl.col('feedback').str.contains('service|quality')
)

customer_id,email,feedback,product_id,order_date,transaction_details
str,str,str,str,str,str
"""C002""","""sarah.connor@shopnow.com""","""Didn't like #product-x2, poor quality. #Product-x…","""P002""","""2023-06-02""","""{""price"": 150.0, ""shipping"": {""type"": ""standard"",…"
"""C004""","""harry.potter@buyonline.org""","""The service was bad. Not happy with #PRODUCT-M1. …","""M001""","""2023-06-04""","""{""price"": 250.0, ""shipping"": {""type"": ""standard"",…"


In [14]:
# filter for products starting with 'M'

df_feedback.filter(
    pl.col('product_id').str.starts_with('M')
)

customer_id,email,feedback,product_id,order_date,transaction_details
str,str,str,str,str,str
"""C004""","""harry.potter@buyonline.org""","""The service was bad. Not happy with #PRODUCT-M1. …","""M001""","""2023-06-04""","""{""price"": 250.0, ""shipping"": {""type"": ""standard"",…"


In [15]:
# Extract tags the feedback column using Regex

df_feedback.select(
    hashtags = pl.col('feedback').str.extract_all(r"(?i)#([\w\d-]+)")
)

hashtags
list[str]
"[""#PRODUCT-X1""]"
"[""#product-x2"", ""#Product-x1""]"
"[""#product-x1"", ""#happy""]"
"[""#PRODUCT-M1"", ""#PRODUCT-M2""]"
"[""#PRODUCT-X1""]"


In [16]:
# Extract tags the feedback column using Regex and convert to lowercase

df_feedback.select(
    hashtags = pl.col('feedback').str.extract_all(r"(?i)#([\w\d-]+)")
        .list.eval(pl.element().str.to_lowercase())
)

hashtags
list[str]
"[""#product-x1""]"
"[""#product-x2"", ""#product-x1""]"
"[""#product-x1"", ""#happy""]"
"[""#product-m1"", ""#product-m2""]"
"[""#product-x1""]"


In [17]:
# keep only 2 characters to hide confidential information

df_feedback.select(
    email_sanitized = pl.col("email").str.replace(r"^(..).*(@.*)$", r"$1$2")
)

email_sanitized
str
"""jo@example.com"""
"""sa@shopnow.com"""
"""ja@onlineshopping.net"""
"""ha@buyonline.org"""
"""li@webshop.com"""


In [18]:
# another way to hide confidential information is to use a hash function
# careful: hashing is vulnerable to a brute force attack so it does not guarantee confidentiality

df_feedback.select(
    email_sanitized = pl.col("email").hash()
)

email_sanitized
u64
9259825263913001176
17451028542328997735
8060747887327259745
11270021373210591822
16256934851045840297


In [19]:
# we can split the email between the username and the domain

df_feedback.select(
    username = pl.col("email").str.split("@").list.get(0),
    domain = pl.col("email").str.split("@").list.get(1)
)

username,domain
str,str
"""john.doe""","""example.com"""
"""sarah.connor""","""shopnow.com"""
"""jack.sparrow""","""onlineshopping.net"""
"""harry.potter""","""buyonline.org"""
"""lisa.simpson""","""webshop.com"""


In [20]:
# we can convert the transaction details column from a JSON to a struct

df_feedback.select(
    transaction_details_as_struct = pl.col("transaction_details").str.json_extract()
)

[/home/runner/work/polars/polars/polars/polars-json/src/ndjson/file.rs:147] &v = [
    Struct(
        [
            Field {
                name: "price",
                data_type: Float64,
                is_nullable: true,
                metadata: {},
            },
            Field {
                name: "shipping",
                data_type: Struct(
                    [
                        Field {
                            name: "type",
                            data_type: LargeUtf8,
                            is_nullable: true,
                            metadata: {},
                        },
                        Field {
                            name: "cost",
                            data_type: Float64,
                            is_nullable: true,
                            metadata: {},
                        },
                    ],
                ),
                is_nullable: true,
                metadata: {},
            },
        ],
    ),

transaction_details_as_struct
struct[2]
"{200.0,{""express"",20.0}}"
"{150.0,{""standard"",10.0}}"
"{200.0,{""express"",20.0}}"
"{250.0,{""standard"",10.0}}"
"{200.0,{""express"",20.0}}"


In [21]:
# once the column is converted, we can access the price and shipping cost

df_feedback.select(
    transaction_details_as_struct = 
        pl.col("transaction_details").str.json_extract()
).select(
    price = 
        pl.col("transaction_details_as_struct").struct.field("price"),
    shipping_cost = 
        pl.col("transaction_details_as_struct").struct.field("shipping")
        .struct.field("cost")
)

[/home/runner/work/polars/polars/polars/polars-json/src/ndjson/file.rs:147] &v = [
    Struct(
        [
            Field {
                name: "price",
                data_type: Float64,
                is_nullable: true,
                metadata: {},
            },
            Field {
                name: "shipping",
                data_type: Struct(
                    [
                        Field {
                            name: "type",
                            data_type: LargeUtf8,
                            is_nullable: true,
                            metadata: {},
                        },
                        Field {
                            name: "cost",
                            data_type: Float64,
                            is_nullable: true,
                            metadata: {},
                        },
                    ],
                ),
                is_nullable: true,
                metadata: {},
            },
        ],
    ),

price,shipping_cost
f64,f64
200.0,20.0
150.0,10.0
200.0,20.0
250.0,10.0
200.0,20.0


#### Time operations

In [22]:
nypd = pl.read_parquet('../datasets/nypd_crimes_from_2018.parquet')

In [23]:
nypd.head(3)

complaint_datetime,complaint_id,precinct,borough,crime_category,crime_description
str,str,i64,str,str,str
"""2019-09-04 09:00:00""","""469069650""",9,"""MANHATTAN""","""MISDEMEANOR""","""LARCENY,PETIT FROM AUTO"""
"""2019-08-31 18:58:00""","""629841380""",50,"""BRONX""","""MISDEMEANOR""","""LARCENY,PETIT OF BICYCLE"""
"""2019-09-07 22:00:00""","""224389328""",19,"""MANHATTAN""","""MISDEMEANOR""","""LARCENY,PETIT FROM STORE-SHOPL"""


In [24]:
# We parse the datetime column to a datetime type

nypd = nypd.with_columns(
    pl.col('complaint_datetime').str.to_datetime("%Y-%m-%d %H:%M:%S")
)

In [25]:
# We can filter for specific dates with the datetime Python package

from datetime import datetime

nypd.filter(
    pl.col('complaint_datetime') >= datetime(2020, 1, 1)
).head(3)

complaint_datetime,complaint_id,precinct,borough,crime_category,crime_description
datetime[μs],str,i64,str,str,str
2020-06-05 20:20:00,"""314830920""",75,null,"""FELONY""",null
2021-05-17 10:00:00,"""473765131""",46,"""BRONX""","""MISDEMEANOR""","""PETIT LARCENY-CHECK FROM MAILB"""
2021-05-29 23:10:00,"""251337835""",63,"""BROOKLYN""","""MISDEMEANOR""","""FORGERY,ETC.-MISD."""


In [26]:
# We can filter for date ranges using the is_between method

nypd.filter(
    pl.col('complaint_datetime')
    .is_between(datetime(2020, 1, 1), datetime(2020, 1, 10))
).head(5)

complaint_datetime,complaint_id,precinct,borough,crime_category,crime_description
datetime[μs],str,i64,str,str,str
2020-01-02 14:45:00,"""591348983""",18,"""MANHATTAN""","""FELONY""","""ROBBERY,OPEN AREA UNCLASSIFIED"""
2020-01-03 15:33:00,"""211105656""",44,"""BRONX""","""MISDEMEANOR""","""LARCENY,PETIT BY CREDIT CARD U"""
2020-01-07 11:20:00,"""364136331""",71,"""BROOKLYN""","""VIOLATION""","""HARASSMENT,SUBD 3,4,5"""
2020-01-02 17:28:00,"""587143439""",42,"""BRONX""","""FELONY""","""CONTROLLED SUBSTANCE,INTENT TO"""
2020-01-03 06:55:00,"""669190911""",50,"""BRONX""","""MISDEMEANOR""","""LEAVING SCENE-ACCIDENT-PERSONA"""


In [27]:
# With the column parsed to a datetime type, we can extract the year, month, day, etc.
# Count of crimes per year

nypd.groupby(
    pl.col('complaint_datetime').dt.year().alias('year')
    ).count()

year,count
i32,u32
2018,462471
2019,458805
2020,413181
2021,448083
2022,520595


In [28]:
# Count of crimes per week of the year, show the 5 weeks with the most crimes

(
    nypd.groupby(
        pl.col('complaint_datetime').dt.week().alias('week'))
    .count()
    .sort('count', descending=True)
    .head(5)
)

week,count
u32,u32
37,47729
38,47481
44,47326
31,47156
27,47117


In [29]:
# There are multiple options for the days: day of the week, day of the month, and day of the year

nypd.with_columns(
    day_of_week = pl.col('complaint_datetime').dt.weekday(), # Monday is 1, Sunday is 7
    day_of_month = pl.col('complaint_datetime').dt.day(),
    day_of_year = pl.col('complaint_datetime').dt.ordinal_day()
).head(5)

complaint_datetime,complaint_id,precinct,borough,crime_category,crime_description,day_of_week,day_of_month,day_of_year
datetime[μs],str,i64,str,str,str,u32,u32,u32
2019-09-04 09:00:00,"""469069650""",9,"""MANHATTAN""","""MISDEMEANOR""","""LARCENY,PETIT FROM AUTO""",3,4,247
2019-08-31 18:58:00,"""629841380""",50,"""BRONX""","""MISDEMEANOR""","""LARCENY,PETIT OF BICYCLE""",6,31,243
2019-09-07 22:00:00,"""224389328""",19,"""MANHATTAN""","""MISDEMEANOR""","""LARCENY,PETIT FROM STORE-SHOPL""",6,7,250
2019-08-31 18:00:00,"""211921838""",71,"""BROOKLYN""","""MISDEMEANOR""","""LARCENY,PETIT FROM AUTO""",6,31,243
2019-09-07 13:00:00,"""381754893""",62,"""BROOKLYN""","""MISDEMEANOR""","""CRIMINAL MISCHIEF 4TH, GRAFFIT""",6,7,250


In [30]:
# Polars has another function to group dates together, which is truncate
# we can bin the dates into buckets of 2 years and 6 months

(
    nypd.groupby(
        pl.col('complaint_datetime').dt.truncate('2y6mo').alias('2year6months_bucket'))
    .count()
)


2year6months_bucket,count
datetime[μs],u32
2020-01-01 00:00:00,1115932
2022-07-01 00:00:00,265927
2017-07-01 00:00:00,921276


In [31]:
# Polars can work with time zones. For example, we can set the time zone to US/Eastern

nypd_localised = nypd.head(100).with_columns(
    complaint_datetime = pl.col('complaint_datetime').dt.replace_time_zone('US/Eastern', use_earliest=True)
)

nypd_localised.head(5)

complaint_datetime,complaint_id,precinct,borough,crime_category,crime_description
"datetime[μs, US/Eastern]",str,i64,str,str,str
2019-09-04 09:00:00 EDT,"""469069650""",9,"""MANHATTAN""","""MISDEMEANOR""","""LARCENY,PETIT FROM AUTO"""
2019-08-31 18:58:00 EDT,"""629841380""",50,"""BRONX""","""MISDEMEANOR""","""LARCENY,PETIT OF BICYCLE"""
2019-09-07 22:00:00 EDT,"""224389328""",19,"""MANHATTAN""","""MISDEMEANOR""","""LARCENY,PETIT FROM STORE-SHOPL"""
2019-08-31 18:00:00 EDT,"""211921838""",71,"""BROOKLYN""","""MISDEMEANOR""","""LARCENY,PETIT FROM AUTO"""
2019-09-07 13:00:00 EDT,"""381754893""",62,"""BROOKLYN""","""MISDEMEANOR""","""CRIMINAL MISCHIEF 4TH, GRAFFIT"""


In [32]:
# When we have a timezone, we can convert it to another timezone

nypd_localised.with_columns(
    complaint_datetime = pl.col('complaint_datetime').dt.convert_time_zone('UTC')
).head(5)

complaint_datetime,complaint_id,precinct,borough,crime_category,crime_description
"datetime[μs, UTC]",str,i64,str,str,str
2019-09-04 13:00:00 UTC,"""469069650""",9,"""MANHATTAN""","""MISDEMEANOR""","""LARCENY,PETIT FROM AUTO"""
2019-08-31 22:58:00 UTC,"""629841380""",50,"""BRONX""","""MISDEMEANOR""","""LARCENY,PETIT OF BICYCLE"""
2019-09-08 02:00:00 UTC,"""224389328""",19,"""MANHATTAN""","""MISDEMEANOR""","""LARCENY,PETIT FROM STORE-SHOPL"""
2019-08-31 22:00:00 UTC,"""211921838""",71,"""BROOKLYN""","""MISDEMEANOR""","""LARCENY,PETIT FROM AUTO"""
2019-09-07 17:00:00 UTC,"""381754893""",62,"""BROOKLYN""","""MISDEMEANOR""","""CRIMINAL MISCHIEF 4TH, GRAFFIT"""


#### Creating a date range and time range

In [33]:
# Polars also allows creating a date range and time range
# the date range method has various parameters including the inteval, whether to include the start and end date, and time zone
# the date_range method can be used to create a table, which is then joined existing data, for example with join_asof (which we'll see later)
# the date_range method can also be used to create lists inside a column of an existing table 

from datetime import date

pl.date_range(date(2020, 1, 1), date(2020, 1, 5), '1d', eager=True)    

date
date
2020-01-01
2020-01-02
2020-01-03
2020-01-04
2020-01-05


In [34]:
pl.date_range(
    date(2020, 1, 1), date(2020, 5, 1), 
    '1mo', eager=True).dt.month_end()    

date
date
2020-01-31
2020-02-29
2020-03-31
2020-04-30
2020-05-31


In [35]:
nypd_localised.select(
    'complaint_datetime',
    pl.date_range(date(2020, 1, 1), date(2020, 5, 1), '1mo', eager=False)
).head(5)

complaint_datetime,date
"datetime[μs, US/Eastern]",list[date]
2019-09-04 09:00:00 EDT,"[2020-01-01, 2020-02-01, … 2020-05-01]"
2019-08-31 18:58:00 EDT,"[2020-01-01, 2020-02-01, … 2020-05-01]"
2019-09-07 22:00:00 EDT,"[2020-01-01, 2020-02-01, … 2020-05-01]"
2019-08-31 18:00:00 EDT,"[2020-01-01, 2020-02-01, … 2020-05-01]"
2019-09-07 13:00:00 EDT,"[2020-01-01, 2020-02-01, … 2020-05-01]"


#### Shifting, differences and percentages: shift, diff, and pct_change

In [36]:
# We can shift rows up or down using shift, calculate difference using diff, and calculate percentage change using pct_change
# for example, we can look at the change in weekly crime counts

(
    nypd.groupby(
        pl.col('complaint_datetime').dt.truncate('1w')
        .cast(pl.Date).alias('week'))
    .count()
    .with_columns(pl.col('count').cast(pl.Int32))
    .sort('week')
    .with_columns(
        prior_week = pl.col('week').shift(1),
        prior_count = pl.col('count').shift(1),
        change = pl.col('count').diff(1),
        percent_change = pl.col('count').pct_change(1).mul(100).round(0))
).head(5)

week,count,prior_week,prior_count,change,percent_change
date,i32,date,i32,i32,f64
2018-01-01,8128,null,null,null,null
2018-01-08,8314,2018-01-01,8128,186,2.0
2018-01-15,8644,2018-01-08,8314,330,4.0
2018-01-22,9020,2018-01-15,8644,376,4.0
2018-01-29,8736,2018-01-22,9020,-284,-3.0


#### Horizontal operations

In [37]:
# Let's start with a dataframe with one column per borough and one row per year

nypd_crimes_by_borough = (
     nypd.with_columns(
        year = pl.col('complaint_datetime').dt.year())
    .pivot(
        index='borough',
        columns= 'year',
        values = 'complaint_id',
        aggregate_function = pl.element().count())
    .filter(pl.col('borough').is_not_null())
    .select('borough', '2018', '2019', '2020', '2021', '2022')
)

nypd_crimes_by_borough

borough,2018,2019,2020,2021,2022
str,u32,u32,u32,u32,u32
"""MANHATTAN""",114201,115595,97145,110296,130973
"""BRONX""",100615,100359,90634,93825,110055
"""BROOKLYN""",136397,131782,119012,125895,144319
"""STATEN ISLAND""",20799,18875,17070,18848,22055
"""QUEENS""",90155,91886,88845,98070,112244


In [38]:
# certain Polars functions calculate the result horizontally if multiple columns are passed 
# one example is sum
# however, not all functions allow it

nypd_crimes_by_borough.select(
    'borough',
    total_crime_count = pl.sum(pl.all().exclude('borough'))
)

borough,total_crime_count
str,u32
"""MANHATTAN""",568210
"""BRONX""",495488
"""BROOKLYN""",657405
"""STATEN ISLAND""",97647
"""QUEENS""",481200


In [39]:
# a general way to calculate expressions horizontally is concatenating the columns as lists
# we can use it with sum, mean, and all functions available in the list namespace

nypd_crimes_by_borough.select(
    'borough',
    avg_crime_count = 
        pl.concat_list(pl.all().exclude('borough')).list.mean()
)

borough,avg_crime_count
str,f64
"""MANHATTAN""",113642.0
"""BRONX""",99097.6
"""BROOKLYN""",131481.0
"""STATEN ISLAND""",19529.4
"""QUEENS""",96240.0


In [40]:
# we can calculate expressions horizontally concatenating the columns as lists

nypd_crimes_by_borough.select(
    'borough',
    median_crime_count = 
        pl.concat_list(pl.all().exclude('borough'))
        .list.eval(pl.element().median()).flatten()
)

borough,median_crime_count
str,f64
"""MANHATTAN""",114201.0
"""BRONX""",100359.0
"""BROOKLYN""",131782.0
"""STATEN ISLAND""",18875.0
"""QUEENS""",91886.0


#### Advanced horizontal operations: reduce and fold

In [41]:
# We reproduce the sum of crimes per borough using reduce

nypd_crimes_by_borough.select(
    'borough',
    total_crime_count = pl.reduce(
        function=lambda accumulator, current: accumulator + current, 
        exprs=pl.all().exclude('borough'))
)

borough,total_crime_count
str,u32
"""MANHATTAN""",568210
"""BRONX""",495488
"""BROOKLYN""",657405
"""STATEN ISLAND""",97647
"""QUEENS""",481200


In [42]:
# a more useful example for reduce is to calculate the maximum crime count per borough

nypd_crimes_by_borough.select(
    'borough',
    max_yearly_crime_count = pl.reduce(
        function=lambda accumulator, current: 
            pl.when(accumulator > current)
            .then(accumulator).otherwise(current),
        exprs=pl.all().exclude('borough'))
)

borough,max_yearly_crime_count
str,u32
"""MANHATTAN""",130973
"""BRONX""",110055
"""BROOKLYN""",144319
"""STATEN ISLAND""",22055
"""QUEENS""",112244


In [43]:
# fold has one additional parameter, which is the initial value of the accumulator
# for example we want to know the number of years where the crime count was above 100,000

nypd_crimes_by_borough.select(
    'borough',
    number_of_years_above_100k = pl.fold(
        acc = pl.lit(0), # initial value of the accumulator 0
        function=lambda accumulator, current: pl.when(current > 100000).then(accumulator + 1).otherwise(accumulator),
        exprs=pl.all().exclude('borough'))
)

borough,number_of_years_above_100k
str,i32
"""MANHATTAN""",4
"""BRONX""",3
"""BROOKLYN""",5
"""STATEN ISLAND""",0
"""QUEENS""",1


In [44]:
# one additional function is the cumfold function, which keeps track of intermediate values and returns a struct
# we can use cumfold to see how the maximum crime count changed over time

nypd_crimes_by_borough.select(
    'borough',
    max_yearly_crime_count = pl.cumfold(
        acc = pl.lit(0), # initial value of the accumulator 0
        function= lambda accumulator, current: 
            pl.when(accumulator > current)
            .then(accumulator).otherwise(current),
        exprs=pl.all().exclude('borough'))
)

borough,max_yearly_crime_count
str,struct[5]
"""MANHATTAN""","{114201,115595,115595,115595,130973}"
"""BRONX""","{100615,100615,100615,100615,110055}"
"""BROOKLYN""","{136397,136397,136397,136397,144319}"
"""STATEN ISLAND""","{20799,20799,20799,20799,22055}"
"""QUEENS""","{90155,91886,91886,98070,112244}"


#### Cumulative functions

In [45]:
sp500 = pl.read_csv('../datasets/SP500.csv', try_parse_dates=True)
btc = pl.read_csv('../datasets/BTC-USD.csv', try_parse_dates=True)

# We know the dates are sorted in the source files, so we can tell Polars about it
# This will make some operations faster 
sp500 = sp500.with_columns(pl.col('Date').set_sorted())
btc = btc.with_columns(pl.col('Date').set_sorted())

In [46]:
sp500.head(5)

Date,Open,High,Low,Close,Adj Close,Volume
date,f64,f64,f64,f64,f64,i64
2019-01-02,2476.96,2519.49,2467.47,2510.03,2510.03,3733160000
2019-01-03,2491.92,2493.14,2443.96,2447.89,2447.89,3858830000
2019-01-04,2474.33,2538.07,2474.33,2531.94,2531.94,4234140000
2019-01-07,2535.61,2566.16,2524.56,2549.69,2549.69,4133120000
2019-01-08,2568.11,2579.82,2547.56,2574.41,2574.41,4120060000


In [47]:
# we can calculate the cumulative max of the adjusted close price

sp500.select(
    'Date','Adj Close',
    cum_max_adj_close = pl.col("Adj Close").cummax()
).head(5)

Date,Adj Close,cum_max_adj_close
date,f64,f64
2019-01-02,2510.03,2510.03
2019-01-03,2447.89,2510.03
2019-01-04,2531.94,2531.94
2019-01-07,2549.69,2549.69
2019-01-08,2574.41,2574.41


In [48]:
# we can add a column with percent change, then calculate the cumulative return with cumprod

sp500.select(
    'Date', 'Adj Close',
    daily_return_percent = pl.col("Adj Close").pct_change().mul(100),
    cumulative_return_percent = 
        pl.col("Adj Close").pct_change()
        .add(1).cumprod().sub(1).mul(100)
).head(5)

Date,Adj Close,daily_return_percent,cumulative_return_percent
date,f64,f64,f64
2019-01-02,2510.03,null,null
2019-01-03,2447.89,-2.475668,-2.475668
2019-01-04,2531.94,3.433569,0.872898
2019-01-07,2549.69,0.701043,1.580061
2019-01-08,2574.41,0.96953,2.56491


In [49]:
# for more advanced calculation, a cumulative_eval function is available
# cumulative_eval calculates a value for each row, by looking the current and all previous rows
# for example, I want to calculate the difference between the current close value and the maximum value to date

sp500.select(
    'Date','Adj Close',
    max_to_date = pl.col('Adj Close').cummax(),
    diff_from_max = pl.col('Adj Close').cumulative_eval(
        pl.element().last() - pl.element().max())
).head(5)

Date,Adj Close,max_to_date,diff_from_max
date,f64,f64,f64
2019-01-02,2510.03,2510.03,0.0
2019-01-03,2447.89,2510.03,-62.14
2019-01-04,2531.94,2531.94,0.0
2019-01-07,2549.69,2549.69,0.0
2019-01-08,2574.41,2574.41,0.0


#### Rolling functions

In [50]:
# Rolling functions are similar to cumulative functions, but we are not limited to looking at all rows
# can specify a number of rows to look at using window_size
# window_size can also be a temporal interval, for example '3d' for 3 days
# the date column should be of type datetime so we can it to datetime first

from datetime import time

(
    sp500.head(5).with_columns(
        pl.col('Date').dt.combine(time(0)))
    .select(
        'Date', 'Adj Close',
        rolling_min_price = 
            pl.col('Adj Close').rolling_min(
                by='Date', window_size='3d', closed='left'))
)

Date,Adj Close,rolling_min_price
datetime[μs],f64,f64
2019-01-02 00:00:00,2510.03,null
2019-01-03 00:00:00,2447.89,2510.03
2019-01-04 00:00:00,2531.94,2447.89
2019-01-07 00:00:00,2549.69,2531.94
2019-01-08 00:00:00,2574.41,2549.69


In [51]:
# we can also use number of rows instead of a temporal interval
# in that case, we pass an integer to window_size
# the result is not the same, because the number of rows is not the same as the number of days

(
    sp500.head(5).select(
        'Date', 'Adj Close',
        rolling_min_price = 
            pl.col('Adj Close').rolling_min(window_size=3))
)

Date,Adj Close,rolling_min_price
date,f64,f64
2019-01-02,2510.03,null
2019-01-03,2447.89,null
2019-01-04,2531.94,2447.89
2019-01-07,2549.69,2447.89
2019-01-08,2574.41,2531.94


In [62]:
# we can join the 2 datasets together to calculate a rolling correlation between the S&P 500 and Bitcoin
import plotly.express

sp500_btc = (
    sp500.select('Date',sp500='Adj Close')
    .join(btc.select('Date', btc='Adj Close'), on='Date', how='inner')
)

(
    sp500_btc
    .with_columns(
        corr = pl.rolling_corr('sp500', 'btc', window_size=30))
    .pipe(plotly.express.line, x='Date', y='corr')
)

In [64]:
# Finally, groupby rolling allows to define custom calculation for each rolling window
# In this example below, we calculate the rolling maximum, and the difference between the current value and the maximum value

(
    sp500.groupby_rolling(
        'Date',
        period='3d',
        closed='left')
    .agg(
        Adj_Close = pl.col('Adj Close').last(),
        rolling_max_price = pl.col('Adj Close').max(),
        rolling_diff_from_max = 
        pl.col('Adj Close').last() - pl.col('Adj Close').max())
).head(5)

Date,Adj_Close,rolling_max_price,rolling_diff_from_max
date,f64,f64,f64
2019-01-02,null,null,null
2019-01-03,2510.03,2510.03,0.0
2019-01-04,2447.89,2510.03,-62.14
2019-01-07,2531.94,2531.94,0.0
2019-01-08,2549.69,2549.69,0.0


#### Dynamic groups: groupby_dynamic

In [66]:
# Groupby_dynamic and groupby_rolling are advanced functions for grouping data
# based on a time interval
# They are different from the cumulative and rolling functions because we reduce the number of rows

# 3 main parameters for groupby_dynamic:
# 1. every: how often we start a new interval
# 2. period: the length of the interval
# 3. offset: the offset of the interval

# Let's calculate the correlation between the S&P 500 and Bitcoin for each month
# Now I calculate only one value per month

(
    sp500_btc
    .groupby_dynamic(
        'Date',
        every='1mo',
        period='1mo')
    .agg(
        from_date = pl.col('Date').min(),
        to_date = pl.col('Date').max(),
        corr = pl.corr('sp500', 'btc'))
).head(5)

Date,from_date,to_date,corr
date,date,date,f64
2019-01-01,2019-01-02,2019-01-31,-0.73916
2019-02-01,2019-02-01,2019-02-28,0.820574
2019-03-01,2019-03-01,2019-03-29,0.599727
2019-04-01,2019-04-01,2019-04-30,0.65675
2019-05-01,2019-05-01,2019-05-31,-0.871049


In [68]:
# Let's now calculate the correlation between the S&P 500 and Bitcoin
# every month, but with a period of 2 months

(
    sp500_btc
    .groupby_dynamic(
        'Date',
        every='1mo',
        period='2mo')
    .agg(
        from_date = pl.col('Date').min(),
        to_date = pl.col('Date').max(),
        corr = pl.corr('sp500', 'btc'))
).head(5)

Date,from_date,to_date,corr
date,date,date,f64
2019-01-01,2019-01-02,2019-02-28,-0.083929
2019-02-01,2019-02-01,2019-03-29,0.840186
2019-03-01,2019-03-01,2019-04-30,0.923523
2019-04-01,2019-04-01,2019-05-31,-0.783081
2019-05-01,2019-05-01,2019-06-28,0.194892


In [69]:
# Finally, let's calculate the correlation between the S&P 500 and Bitcoin
# every 2 months, but with a period of 1 month

(
    sp500_btc
    .groupby_dynamic(
        'Date',
        every='2mo',
        period='1mo')
    .agg(
        from_date = pl.col('Date').min(),
        to_date = pl.col('Date').max(),
        corr = pl.corr('sp500', 'btc'))
).head(5)

Date,from_date,to_date,corr
date,date,date,f64
2019-01-01,2019-01-02,2019-01-31,-0.73916
2019-03-01,2019-03-01,2019-03-29,0.599727
2019-05-01,2019-05-01,2019-05-31,-0.871049
2019-07-01,2019-07-01,2019-07-31,-0.365423
2019-09-01,2019-09-03,2019-09-30,0.081172


#### Join with nearest key: asof_join

In [70]:
# Join_asof is a function that allows to join 2 datasets based on nearest keys
# For example, we can find the price of purchase for an S&P 500 ETF by looking at opening price for the next available date

trades = pl.DataFrame({
    'Date': [date(2019, 1, 4), date(2019, 1, 6), date(2019, 1, 11)],
    'Quantity': [10, -20, 30]
}).with_columns(
    pl.col('Date').set_sorted()
)

trades

Date,Quantity
date,i64
2019-01-04,10
2019-01-06,-20
2019-01-11,30


In [72]:
# Here are the prices of the S&P 500 as a reminder
btc.select('Date', 'Open').head(10)

Date,Open
date,f64
2019-01-01,3746.713379
2019-01-02,3849.216309
2019-01-03,3931.048584
2019-01-04,3832.040039
2019-01-05,3851.973877
2019-01-06,3836.519043
2019-01-07,4078.584961
2019-01-08,4028.472168
2019-01-09,4031.552002


In [73]:
trades.join_asof(
    btc.select('Date', Price = 'Open'), 
    on='Date', 
    strategy='forward')

Date,Quantity,Price
date,i64,f64
2019-01-04,10,3832.040039
2019-01-06,-20,3836.519043
2019-01-11,30,3674.015381
